# LangChain-Powered MedTech Assistant Notebook

## Overview
This notebook implements a LangChain-powered MedTech assistant with the following capabilities:

###  Smart Query Classification
Automatically categorizes input as one of:
- **DB-related**: Searches for known medical innovations.
- **Idea**: Recognizes user-proposed MedTech concepts.
- **Irrelevant**: Filters out non-MedTech content.

###  Semantic Search with FAISS
Uses sentence-transformer embeddings and a FAISS vector index to retrieve similar MedTech innovations from a mock database.

###  Clarification for Idea Refinement
When a user submits a new idea, the assistant:
- Retrieves related innovations.
- Asks 2–3 clarifying questions to improve the idea.

###  LLM-Powered Fallbac k  Answering
If a query doesn’t match existing DB entries, the assistant leverages an LLM to provide a helpful general explanation.

###  Conversation Memory + Summarization
- Tracks chat history using `ConversationBufferMemory`.
- Summarizes recent exchanges to provide continuity.

of your script automatically?



### Pre-Requisites 

We Installed LLAMA3 for  API calls/LLM calls.You can simply run "ollama run llama3" in terminal and it should be working fine

In [5]:
!pip install langchain langchain-community
!pip install rapidfuzz


In [ ]:
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from rapidfuzz import fuzz
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# === Embedding Model ===
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

#Here we are creating  a MOCK Data base so we can show the logic of  when to come for DB and When to seek LLM

innovation_db = [
    {"id": 1, "title": "Smart Pulse Oximeter", "description": "Measures oxygen saturation and heart rate with smartphone connectivity."},
    {"id": 2, "title": "Portable ECG Monitor", "description": "A compact device for real-time ECG monitoring at home."},
    {"id": 3, "title": "Wireless Glucometer", "description": "Measures blood glucose levels and syncs data with cloud platforms."},
    {"id": 4, "title": "Digital Stethoscope", "description": "Amplifies and records heart and lung sounds digitally."},
    {"id": 5, "title": "AI-Powered Thermometer", "description": "Uses machine learning to predict fever patterns and trends."},
    {"id": 6, "title": "Wearable Blood Pressure Monitor", "description": "Continuous BP monitoring via a wrist-worn device."},
    {"id": 7, "title": "Smart Inhaler", "description": "Tracks asthma medication usage and sends reminders."},
    {"id": 8, "title": "Handheld Ultrasound Scanner", "description": "Portable ultrasound imaging for bedside diagnostics."},
    {"id": 9, "title": "Remote Fetal Monitor", "description": "Monitors fetal heart rate and movement remotely during pregnancy."},
    {"id": 10, "title": "Digital Otoscope", "description": "Captures inner ear images and transmits them for remote diagnosis."},
    {"id": 11, "title": "Smart Nebulizer", "description": "Delivers aerosol medication with dose tracking and mobile alerts."},
    {"id": 12, "title": "AI-Based Retinal Scanner", "description": "Detects early signs of diabetic retinopathy through eye scans."},
    {"id": 13, "title": "Connected Spirometer", "description": "Measures lung function and tracks respiratory performance over time."},
    {"id": 14, "title": "Digital Pill Dispenser", "description": "Ensures correct medication timing with alerts and tracking."},
    {"id": 15, "title": "Smart IV Pump", "description": "Adjusts infusion rate based on patient vitals and alerts caregivers."},
    {"id": 16, "title": "Smart Hearing Aid", "description": "Adapts to environment noise levels and connects to smartphones for customization."},
    {"id": 17, "title": "Smart Contact Lens", "description": "Measures intraocular pressure and monitors diabetes via tear glucose levels."},
    {"id": 18, "title": "Digital Wound Monitor", "description": "Uses imaging and sensors to monitor wound healing progress and infection risk."},
    {"id": 19, "title": "Digital X-ray Machine", "description": "Captures internal bone and organ images using low-dose X-ray radiation for diagnosis."}
]

# === FAISS Index Build ===
record_texts = [f"{rec['title']} {rec['description']}" for rec in innovation_db]
record_embeddings = embedding_model.encode(record_texts, convert_to_numpy=True)
index = faiss.IndexFlatL2(record_embeddings.shape[1])
index.add(record_embeddings)

# For Models we used Local LLAMA3 model.
llm = Ollama(model="llama3")

# === Memory ===
# Usually memory is needed for context , but having a large K will create issues of computation so we are summarizing the texts  before going forward.

buffer_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# === Summarization Chain ===
summary_prompt = PromptTemplate.from_template(
    """You are a helpful assistant. Summarize the following conversation so far:

{chat_history}

Summary:"""
)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

def get_summary():
    history = buffer_memory.chat_memory.messages[-10:]  # Only last 5 user+AI turns
    return summary_chain.run(chat_history=history)

# === Classifier Chain ===
classifier_prompt = PromptTemplate.from_template(
    """
You are a strict classifier assistant. Categorize the user query strictly as one of the following:

- DB-related: If the query asks about or searches innovation records or known technologies such as pulse oximeter, pulse meter, ECG monitor, glucometer, digital stethoscope, AI thermometer, blood pressure monitor, smart inhaler, ultrasound scanner, fetal monitor, otoscope, smart nebulizer, retinal scanner, spirometer, pill dispenser, IV pump, hearing aid, contact lens, wound monitor, X-ray machine, or other medical devices and health monitors.

- Idea: If the query proposes a new concept or idea to improve a process or product.

- Irrelevant: If the query is general knowledge (e.g. what is RAM, history of India), or does not relate to MedTech or innovation.

Be strict. Do not guess. Use only the content of the query.

Query: {query}

Classification:
"""
)


classifier_chain = LLMChain(llm=llm, prompt=classifier_prompt)

def classify_query(query: str) -> str:
    try:
        raw_result = classifier_chain.run(query=query).strip().lower().splitlines()[0]
        if "db" in raw_result:
            return "db-related"
        elif "idea" in raw_result:
            return "idea"
        else:
            return "irrelevant"
    except Exception:
        return "irrelevant"

# === Vector Search ===
def vector_db_search(query: str, top_k: int = 3, threshold: float = 0.75):
    query_embedding = embedding_model.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, top_k)
    results = []
    for i, dist in zip(I[0], D[0]):
        if dist < threshold:
            results.append(innovation_db[i])
    return results

def find_similar_use_cases(idea_text, top_k=3):
    return vector_db_search(idea_text, top_k=top_k)

# === Clarifying Questions ===
clarify_prompt = PromptTemplate.from_template(
    """You are an innovation expert. Read the idea carefully and generate 2-3 clarifying questions that would help improve it.

Idea: {idea}

Clarifying Questions:"""
)
clarify_chain = LLMChain(llm=llm, prompt=clarify_prompt)

def get_clarifying_questions(idea: str) -> str:
    return clarify_chain.run(idea=idea).strip()

# === General Fallback LLM Answer Chain ===
fallback_prompt = PromptTemplate.from_template(
    """The user asked: {query}
    
Before providing your final answer, first explain your step-by-step reasoning and considerations to arrive at the answer in a sperate paragraph.Once you hvae
provided the reasoning then only provide the response.

Provide a helpful and informative response:"""
    
)
llm_chain = LLMChain(llm=llm, prompt=fallback_prompt)

# === Handle Query ===
def handle_query(user_query: str) -> str:
    category = classify_query(user_query)

    if category == "db-related":
        records = vector_db_search(user_query)
        if records:
            return "Innovation DB Records Found:\n" + "\n".join(
                [f"- {r['title']}: {r['description']}" for r in records]
            )
        else:
            return llm_chain.run(query=user_query)

    elif category == "idea":
        similar = find_similar_use_cases(user_query)
        response = "Similar Use Cases:\n"
        if similar:
            response += "\n".join([f"- {s['title']}: {s['description']}" for s in similar])
        else:
            response += "No similar use cases found."

        questions = get_clarifying_questions(user_query)
        response += "\n\nTo refine your idea, consider these questions:\n" + questions
        return response

    else:
        return "Sorry, this assistant only handles MedTech-related queries. Please ask about medical devices, healthcare innovations, or propose an idea in that domain."

# === CLI Interaction with Summary ===
if __name__ == "__main__":
    print("Welcome! Type your query below (type 'exit' to quit).")
    while True:
        user_input = input("\nYour Query: ").strip()
        if user_input.lower() == "exit":
            break
        if user_input.lower() in {"help", "options"}:
            print("You can ask about known MedTech devices, propose an idea, or ask general MedTech questions.")
            continue

        buffer_memory.chat_memory.add_user_message(user_input)

        reply = handle_query(user_input)

        buffer_memory.chat_memory.add_ai_message(reply)

        print("\n--- Response ---")
        print(reply)

        summary = get_summary()
        print("\n--- Summary So Far ---")
        print(summary)



Welcome! Type your query below (type 'exit' to quit).



Your Query:  Can you help me with Pulse Meter ?



--- Response ---
Here's my step-by-step reasoning and considerations:

When considering how to assist with a Pulse Meter, I think about the various aspects that are involved. A pulse meter is typically used to measure a person's heart rate or blood flow in real-time. To provide assistance, I would need to understand what the user means by "Pulse Meter". Are they referring to a physical device, such as a wearable fitness tracker or a medical device used in hospitals? Or are they asking about how to read and interpret pulse data from a digital display? Additionally, I would want to know if the user has any specific goals or concerns related to their heart rate or blood flow. For instance, are they trying to monitor their physical activity levels, manage stress, or diagnose a medical condition? By considering these factors, I can provide more tailored and accurate guidance.

Now, here's my response:

To help you with your Pulse Meter query, I'd like to clarify a few things. Can you pleas


Your Query:  Can you help me with Python Programming ?



--- Response ---
Sorry, this assistant only handles MedTech-related queries. Please ask about medical devices, healthcare innovations, or propose an idea in that domain.

--- Summary So Far ---
The conversation started with a human message asking for help with "Pulse Meter". The AI responded by providing a step-by-step reasoning and considerations to understand what the user means by "Pulse Meter", which includes clarifying whether it refers to a physical device, digital display, or specific goals/concerns related to heart rate or blood flow. The AI then asked follow-up questions to gather more information from the user.

However, the conversation took a turn when the human sent another message asking for help with "Python Programming". The AI responded by stating that it only handles MedTech-related queries and does not assist with general programming topics like Python.


## Response Recieved from the CODE 





Welcome! Type your query below (type 'exit' to quit).

Your Query:  Can you help me with Pulse Meter ?

--- Response ---
Here's my step-by-step reasoning and considerations:

When considering how to assist with a Pulse Meter, I think about the various aspects that are involved. A pulse meter is typically used to measure a person's heart rate or blood flow in real-time. To provide assistance, I would need to understand what the user means by "Pulse Meter". Are they referring to a physical device, such as a wearable fitness tracker or a medical device used in hospitals? Or are they asking about how to read and interpret pulse data from a digital display? Additionally, I would want to know if the user has any specific goals or concerns related to their heart rate or blood flow. For instance, are they trying to monitor their physical activity levels, manage stress, or diagnose a medical condition? By considering these factors, I can provide more tailored and accurate guidance.

Now, here's my response:

To help you with your Pulse Meter query, I'd like to clarify a few things. Can you please specify what kind of Pulse Meter you're referring to? Are you looking for information on how to use a specific device, or do you have questions about interpreting pulse data? Additionally, are there any specific goals or concerns you have related to your heart rate or blood flow that I can help with? With this information, I'll be happy to provide more targeted assistance.

--- Summary So Far ---
There is no conversation yet, as the first message from a human user asking for help with "Pulse Meter" has just been received. The AI assistant responded by providing a step-by-step reasoning and considerations on how to assist with a Pulse Meter, and then asked clarifying questions to understand the user's query better.

Your Query:  Can you help me with Python Programming ?

--- Response ---
Sorry, this assistant only handles MedTech-related queries. Please ask about medical devices, healthcare innovations, or propose an idea in that domain.

--- Summary So Far ---
The conversation started with a human message asking for help with "Pulse Meter". The AI responded by providing a step-by-step reasoning and considerations to understand what the user means by "Pulse Meter", which includes clarifying whether it refers to a physical device, digital display, or specific goals/concerns related to heart rate or blood flow. The AI then asked follow-up questions to gather more information from the user.

However, the conversation took a turn when the human sent another message asking for help with "Python Programming". The AI responded by stating that it only handles MedTech-related queries and does not assist with general programming topics like Python.

Your Query: 
↑↓ for history. Search history with c-↑/c-↓
